# Introdução

Segundo o site https://developers.mercadolivre.com.br/pt_br/boas-praticas-para-usar-a-plataforma, é recomendado utilizar a API do Mercado Livre em vez de fazer web crawling, portanto foi o caminho escolhido para coletar informações de produtos.

Para funcionar as chamadas, é preciso criar uma conta em developers.mercadolivre.com.br, criar um aplicativo, e conseguir um token de acesso. Esse token deve ser colocar no arquivo .env na variável "ML_ACCESS_TOKEN".

IDs de categorias obtidas no site de documentação da API do Mercado Livre:

In [6]:
import pandas as pd
import requests
import os

ML_ACCESS_TOKEN = os.getenv("ML_ACCESS_TOKEN")

headers = {
    "Authorization": f"Bearer {ML_ACCESS_TOKEN}"
}
url = "https://api.mercadolibre.com/sites/MLB/categories"
resp = requests.get(url, headers=headers)
print(resp.json())

[{'id': 'MLB5672', 'name': 'Acessórios para Veículos'}, {'id': 'MLB271599', 'name': 'Agro'}, {'id': 'MLB1403', 'name': 'Alimentos e Bebidas'}, {'id': 'MLB1071', 'name': 'Animais'}, {'id': 'MLB1367', 'name': 'Antiguidades e Coleções'}, {'id': 'MLB1368', 'name': 'Arte, Papelaria e Armarinho'}, {'id': 'MLB1384', 'name': 'Bebês'}, {'id': 'MLB1246', 'name': 'Beleza e Cuidado Pessoal'}, {'id': 'MLB1132', 'name': 'Brinquedos e Hobbies'}, {'id': 'MLB1430', 'name': 'Calçados, Roupas e Bolsas'}, {'id': 'MLB1039', 'name': 'Câmeras e Acessórios'}, {'id': 'MLB1743', 'name': 'Carros, Motos e Outros'}, {'id': 'MLB1574', 'name': 'Casa, Móveis e Decoração'}, {'id': 'MLB1051', 'name': 'Celulares e Telefones'}, {'id': 'MLB1500', 'name': 'Construção'}, {'id': 'MLB5726', 'name': 'Eletrodomésticos'}, {'id': 'MLB1000', 'name': 'Eletrônicos, Áudio e Vídeo'}, {'id': 'MLB1276', 'name': 'Esportes e Fitness'}, {'id': 'MLB263532', 'name': 'Ferramentas'}, {'id': 'MLB12404', 'name': 'Festas e Lembrancinhas'}, {'id':

In [7]:
cat_df = pd.DataFrame(resp.json())

In [9]:
cat_df.head()

,id,name
0,MLB5672,Acessórios para Veículos
1,MLB271599,Agro
2,MLB1403,Alimentos e Bebidas
3,MLB1071,Animais
4,MLB1367,Antiguidades e Coleções
